In [1]:
import simpy
from random import expovariate, seed

In [2]:
# global constants
N_PROCESSORS = 1
N_REQUESTS = 2000
MAX_QUEUE_LEN = 100
AVG_REQUEST_INTERVAL = 1000
AVG_SERVING_TIME = 900
MAX_TIME = 3000000
SEED = 42

# monitoring variables
PROC_WORKING_TIME = 0
SERVICE_TIME = []
WAITING_TIME = []

In [3]:
def generate_requests(n_jobs, avg_interval, resource):
    for i in range(n_jobs):
        env.process(serve(i, resource))
        t = expovariate(1.0/avg_interval)
        yield env.timeout(t) # wait for the next request to appear      

In [4]:
def serve(job_id, resource):
    num_rejected = 0
    arrive = env.now
    if len(resource.queue) < MAX_QUEUE_LEN:     # the test
        with resource.request() as req:
            yield req
            wait_time = env.now-arrive
            WAITING_TIME.append(wait_time)
            
            service_time = expovariate(1.0/AVG_SERVING_TIME)
            SERVICE_TIME.append(service_time)
            yield env.timeout(service_time) # wait to be served
            # saved here for debug purposes
        
        global PROC_WORKING_TIME
        PROC_WORKING_TIME += env.now - arrive 
#         print("job_id={}, start={}, finish={}".format(job_id, arrive, env.now))
    else:
        num_rejected += 1
        print("%8.4f: BALKING   "%(env.now))

In [5]:
seed(SEED)

# GLOBAL VAR MEASURING PROC WORKLOAD

env = simpy.Environment()
k = simpy.Resource(env, capacity=N_PROCESSORS)
env.process(generate_requests(n_jobs=N_REQUESTS, 
                              avg_interval=AVG_REQUEST_INTERVAL, 
                              resource=k))
env.run(until=MAX_TIME)
print(PROC_WORKING_TIME/MAX_TIME)

3.09301143791507


In [6]:
print(len(WAITING_TIME),  len(SERVICE_TIME))
print(sum(SERVICE_TIME) / len(SERVICE_TIME))

2000 2000
885.9054979755136
